In [ ]:
from pathlib import Path

import tensorflow as tf
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt

from src.models.fetch_data_from_hdf5 import get_tf_data
from src.models.models_2d import CustomModel, unet_model

In [ ]:

project_dir = Path("../").resolve()
#model_path = "/home/val/python_wkspce/plc_seg/models/model_1-00_alpha_1-00_wplc_1-00_wt_1-00_wl_10-00_splc_early_stop/checkpoint/checkpoint"
#model_path = "/home/val/python_wkspce/plc_seg/models/model_0-75_alpha_1-00_wplc_1-00_wt_1-00_wl_40-00_splc_early_stop/epochs_50"
model_path = "/home/val/python_wkspce/plc_seg/models/early_stop/model_0-75_alpha_1-00_wplc_0-00_wt_0-00_wl_40-00_splc_early_stop/final"
image_size = (256, 256)


In [ ]:
model_path

In [ ]:
file_test = h5py.File("/home/val/python_wkspce/plc_seg/data/processed/2d_pet_normalized/test.hdf5", "r")
file_train = h5py.File("/home/val/python_wkspce/plc_seg/data/processed/2d_pet_normalized/train.hdf5", "r")

In [ ]:
clinical_df = pd.read_csv("../data/clinical_info.csv").set_index("patient_id")

In [ ]:
bs = 4
data_test = get_tf_data(
        file_test,
        clinical_df,
        output_shape=(256, 256),
        random_slice=False,
    ).batch(bs).as_numpy_iterator()

In [ ]:
#model_ = unet_model(3, input_shape=image_size + (3, ))
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

#model = CustomModel(model_.inputs, model_.outputs)
# We don't passs a loss or metrics here.



In [ ]:
model = tf.keras.models.load_model(model_path, compile=False)

In [ ]:
model.compile(
    optimizer=optimizer,
    run_eagerly=False,
)

In [ ]:
#model.load_weights(model_path)


In [ ]:
bs = 4
data_train = get_tf_data(
        file_train,
        clinical_df,
        output_shape=(224, 224),
        random_slice=False,
    ).batch(bs)

In [ ]:
x, y_true, plc_status, sick_lung_axis = next(data_test)

In [ ]:
y_pred = model(x)

In [ ]:
np.mean(x[b,:,:,1])

In [ ]:
b=3
print(f"PLC status {plc_status[b]}")
fig = plt.figure(figsize=(4, 4)) 
#plt.imshow(x[b,:,:,0],cmap='gray')
#plt.imshow(y_pred[b, :,:,1], cmap='jet', alpha=0.5)

ax = plt.axes()
im = ax.imshow(x[b,:,:,0],cmap="gray")
im = ax.imshow(y_pred[b, :,:,1], cmap='jet', alpha=0.5)
cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
plt.colorbar(im, cax=cax) # Similar to fig.colorbar(im, cax = cax)

# plt.savefig("../reports/plc_pos_success.png")


In [ ]:
plt.figure(figsize=(4, 4))
plt.imshow(x[b,:,:,0],cmap='gray')
plt.imshow(y_true[b, :,:,1], cmap='jet', alpha=0.5)
# plt.savefig("../reports/plc_pos_success_gt.png")

In [ ]:
x_train, y_true_train, plc_status_train, sick_lung_axis = next(data_train.as_numpy_iterator())
y_pred_train = model(x_train)
y_pred_b_train = np.copy(y_pred_train)
y_pred_b_train[y_pred_b_train>=0.5] = 1
y_pred_b_train[y_pred_b_train<0.5] = 0


In [ ]:
b=0
print(f"PLC status {plc_status[b]}")
plt.imshow(x_train[b,:,:,0],cmap='gray')
plt.imshow(y_pred_train[b, :,:,0], cmap='jet', alpha=0.5)

In [ ]:
plt.imshow(x_train[b,:,:,0],cmap='gray')
plt.imshow(y_true_train[b, :,:,0], cmap='jet', alpha=0.5)